In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import networkx
import h5py
# import obonet
from Bio import SeqIO
import re
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import gc
import pickle
import psutil
import torch
import joblib
#from transformers import T5EncoderModel, T5Tokenizer
# from tape import ProteinBertModel, UniRepModel, TAPETokenizer

#### Getting the protein sequence from fasta files

In [2]:
id = []
seq = [] 

for seq_record in SeqIO.parse(".../train_sequences.fasta", "fasta"):
  id.append(seq_record.id)
  seq.append(str(seq_record.seq))

print(len(id))

142246


In [3]:
seq_df = pd.DataFrame({"id": id, "sequence": seq})
del id
del seq
seq_df.head()

,id,sequence
0,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...
1,O73864,MTEYRNFLLLFITSLSVIYPCTGISWLGLTINGSSVGWNQTHHCKL...
2,O95231,MRLSSSPPRGPQQLSSFGSVDWLSQSSCSGPTHTPRPADFSLGSLP...
3,A0A0B4J1F4,MGGEAGADGPRGRVKSLGLVFEDESKGCYSSGETVAGHVLLEAAEP...
4,P54366,MVETNSPPAGYTLKRSPSDLGEQQQPPRQISRSPGNTAAYHLTTAM...


In [4]:
gc.collect()

0

In [5]:
seq_df = seq_df.sort_values("id").reset_index().drop("index", axis = 1)
seq_df.head()

,id,sequence
0,A0A009IHW8,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
1,A0A021WW32,MFYEHIILAKKGPLARIWLAAHWDKKITKAHVFETNIEKSVEGILQ...
2,A0A021WZA4,MKYINCTQPAIDDFPRDLFSEAQRQSGAVVLHVIASLYLFVALAVV...
3,A0A023FBW4,MTSHGAVKIAIFAVIALHSIFECLSKPQILQRTDHSTDSDWDPQMC...
4,A0A023FBW7,MKVLLYIAASCLMLLALNVSAENTQQEEEDYDYGTDTCPFPVLANK...


#### Extract other features

In [6]:
lst = []
for s in seq_df["sequence"]:
  lst.append(ProteinAnalysis(s).get_amino_acids_percent())
X = pd.DataFrame(lst)
X.head()

,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
0,0.040892,0.000000,0.074349,0.085502,0.014870,0.029740,0.018587,0.096654,0.126394,0.107807,0.011152,0.055762,0.022305,0.055762,0.033457,0.081784,0.059480,0.048327,0.011152,0.026022
1,0.061611,0.006319,0.078989,0.082148,0.045814,0.058452,0.033175,0.067930,0.056872,0.091627,0.025276,0.067930,0.048973,0.030016,0.036335,0.082148,0.045814,0.056872,0.006319,0.017378
2,0.110220,0.032064,0.038076,0.038076,0.052104,0.076152,0.012024,0.086172,0.042084,0.094188,0.030060,0.038076,0.036072,0.022044,0.020040,0.068136,0.070140,0.088176,0.016032,0.030060
3,0.061856,0.072165,0.082474,0.041237,0.030928,0.092784,0.051546,0.061856,0.030928,0.061856,0.030928,0.041237,0.041237,0.041237,0.010309,0.113402,0.072165,0.041237,0.020619,0.000000
4,0.059322,0.076271,0.059322,0.093220,0.016949,0.076271,0.008475,0.008475,0.101695,0.093220,0.033898,0.059322,0.033898,0.033898,0.033898,0.025424,0.059322,0.067797,0.016949,0.042373


In [7]:
del lst

In [8]:
train_terms = pd.read_table(".../train_terms.tsv")
train_terms.head()

,EntryID,term,aspect
0,A0A009IHW8,GO:0008152,BPO
1,A0A009IHW8,GO:0034655,BPO
2,A0A009IHW8,GO:0072523,BPO
3,A0A009IHW8,GO:0044270,BPO
4,A0A009IHW8,GO:0006753,BPO


In [9]:
# get most occured label
num_label = 1000
#1500 => 0.24 micro f1-score
#3000 => 0.21 micro f1-score
#1000 => 0.26

freqCount = (train_terms['term'].value_counts())
print(freqCount)
considered_one = list(freqCount.index[:num_label])

GO:0005575    92912
GO:0008150    92210
GO:0110165    91286
GO:0003674    78637
GO:0005622    70785
              ...  
GO:0031772        1
GO:0042324        1
GO:0031771        1
GO:0051041        1
GO:0102628        1
Name: term, Length: 31466, dtype: int64


In [10]:
# check if these features are enough to cover all cases
train_terms[train_terms["term"].isin(considered_one)]["EntryID"].nunique() - 142246 # need to be 1
# good for case of 1000+

0

In [11]:
# make multilabel data
train_size = len(seq_df)
Y = np.zeros((train_size ,num_label))
train_protein = pd.Series(seq_df["id"])
train_terms_smaller = train_terms[train_terms["term"].isin(considered_one)]
for i in tqdm(range(Y.shape[1])):
    m = train_terms_smaller['term'] ==  considered_one[i]
    Y[:,i] =  train_protein.isin( set(train_terms_smaller[m]['EntryID'] ) ).astype(float )
Y

100%|██████████| 1000/1000 [05:23<00:00,  3.09it/s]


array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.]])

In [12]:
Y.shape

(142246, 1000)

In [13]:
X = X.values
X.shape

(142246, 20)

In [14]:
del train_terms
del freqCount

#### Train using current features

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 101)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [16]:
from sklearn.metrics import f1_score

Binary Relevance Naive Bayes

In [17]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

In [18]:
br_gnb = BinaryRelevance(GaussianNB())

In [19]:
br_gnb.fit(X_train, Y_train)
# 25 min to train if use embedding
# 1 min to train if use protein occurence

BinaryRelevance(classifier=GaussianNB(), require_dense=[True, True])

In [20]:
pred_br_gnb = br_gnb.predict(X_test)
# 19 min to test if use embedding
# 1 min to test if use protein occurence

In [21]:
print(f1_score(Y_test,pred_br_gnb.toarray(), average="micro")) 
print(f1_score(Y_test,pred_br_gnb.toarray(), average="macro"))

0.2636397568191104
0.07028354903514487


In [22]:
del X
del Y
del X_train
del Y_train
del X_test
del Y_test
del pred_br_gnb
del seq_df

Try on test data

In [23]:
test_id = []
test_seq = []

for seq_record in SeqIO.parse(".../testsuperset.fasta", "fasta"):
  test_id.append(seq_record.id)
  test_seq.append(str(seq_record.seq))

print(len(test_id))

141865


In [24]:
test_df = pd.DataFrame({"id": test_id, "sequence": test_seq})
del test_id
del test_seq
test_df.head()

,id,sequence
0,Q9CQV8,MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLL...
1,P62259,MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLS...
2,P68510,MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLS...
3,P61982,MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLS...
4,O70456,MERASLIQKAKLAEQAERYEDMAAFMKSAVEKGEELSCEERNLLSV...


In [25]:
test_df = test_df.sort_values("id").reset_index().drop("index", axis = 1)
test_df.head()

,id,sequence
0,A0A023PXA5,MLLSELVATASSLPYTAISIHNNCRVPAARHIHHGCRYFHGPPVMH...
1,A0A023PXB0,MFINGFVNYPVRTPPNDLLQVVLHGFLRCPLDGSQVDSIGIGHTVH...
2,A0A023PXB5,MFALIISSKGKTSGFFFNSSFSSSALVGIAPLTAYSALVTPVFKSF...
3,A0A023PXB9,MEYVLIYNIWFFSFLQDKPCFCFVDYACSIFLLSSYCGNCLTAVAT...
4,A0A023PXC2,MLPLCLTFLSFFLSLGGSFKAVMTKEEADGTTEAAACLFWIFNWTV...


In [26]:
id_lst = test_df["id"].tolist()

Make test data and remove all those unneeded or write the needed into disk in order to prevent ram overloading

In [27]:
test_lst = []
for s in test_df["sequence"]:
  test_lst.append(ProteinAnalysis(s).get_amino_acids_percent())
test = pd.DataFrame(test_lst)
test = test.values

In [28]:
del test_df
del test_lst

In [29]:
test.shape

(141865, 20)

In [30]:
test_sub1 = test[:30000]
test_sub2 = test[30000:60000]
test_sub3 = test[60000:90000]
test_sub4 = test[90000:120000]
test_sub5 = test[120000:]
del test

In [31]:
prob_1 = br_gnb.predict(test_sub1)
del test_sub1
with open('prob_1.pickle', 'wb') as f:
    pickle.dump(prob_1, f)
del prob_1

In [32]:
gc.collect()

0

In [33]:
prob_2 = br_gnb.predict_proba(test_sub2)
del test_sub2
with open('prob_2.pickle', 'wb') as f:
    pickle.dump(prob_2, f)
del prob_2

In [34]:
gc.collect()

0

In [35]:
prob_3 = br_gnb.predict_proba(test_sub3)
del test_sub3
with open('prob_3.pickle', 'wb') as f:
    pickle.dump(prob_3, f)
del prob_3

In [36]:
gc.collect()

0

In [37]:
prob_4 = br_gnb.predict_proba(test_sub4)
del test_sub4
with open('prob_4.pickle', 'wb') as f:
    pickle.dump(prob_4, f)
del prob_4

In [38]:
gc.collect()

0

In [39]:
prob_5 = br_gnb.predict_proba(test_sub5)
del test_sub5
with open('prob_5.pickle', 'wb') as f:
    pickle.dump(prob_5, f)
del prob_5

In [40]:
gc.collect()

0

Now concat the data with the label

In [41]:
final_df = pd.DataFrame(columns = ["id", "terms", "prob"])

In [42]:
final_df["id"] = [id_lst[i] for i in range(30000) for _ in range(1000)]

In [43]:
final_df["terms"] = considered_one * 30000

In [44]:
with open('prob_1.pickle', 'rb') as f:
    prob_1 = pickle.load(f)
prob_1.shape

(30000, 1000)

In [45]:
prob_1 = prob_1.toarray().ravel()

In [46]:
final_df["prob"] = prob_1

In [47]:
del prob_1

In [48]:
final_df = final_df[final_df["prob"] >= 0.6]

In [49]:
final_df.shape

(889389, 3)

In [50]:
gc.collect()

13

In [51]:
temp_df = pd.DataFrame(columns = ["id", "terms", "prob"])

In [52]:
temp_df["id"] = [id_lst[i] for i in range(30000, 60000) for _ in range(1000)]

In [53]:
temp_df["terms"] = considered_one * 30000

In [54]:
with open('prob_2.pickle', 'rb') as f:
    prob_2 = pickle.load(f)

In [55]:
prob_2 = prob_2.toarray().ravel()

In [56]:
temp_df["prob"] = prob_2

In [57]:
del prob_2

In [58]:
temp_df = temp_df[temp_df["prob"] >= 0.6]

In [59]:
temp_df.shape

(597785, 3)

In [60]:
final_df = pd.concat([final_df, temp_df])
del temp_df
final_df = final_df.reset_index().drop("index", axis = 1)

In [61]:
gc.collect()

0

In [62]:
temp_df = pd.DataFrame(columns = ["id", "terms", "prob"])

In [63]:
temp_df["id"] = [id_lst[i] for i in range(60000, 90000) for _ in range(1000)]

In [64]:
temp_df["terms"] = considered_one * 30000

In [65]:
with open('prob_3.pickle', 'rb') as f:
    prob_3 = pickle.load(f)

In [66]:
prob_3 = prob_3.toarray().ravel()

In [67]:
temp_df["prob"] = prob_3

In [68]:
del prob_3

In [69]:
temp_df = temp_df[temp_df["prob"] >= 0.6]

In [70]:
temp_df.shape

(658131, 3)

In [71]:
final_df = pd.concat([final_df, temp_df])
del temp_df
final_df = final_df.reset_index().drop("index", axis = 1)

In [72]:
gc.collect()

0

In [73]:
temp_df = pd.DataFrame(columns = ["id", "terms", "prob"])

In [74]:
temp_df["id"] = [id_lst[i] for i in range(90000, 120000) for _ in range(1000)]

In [75]:
temp_df["terms"] = considered_one * 30000

In [76]:
with open('prob_4.pickle', 'rb') as f:
    prob_4 = pickle.load(f)

In [77]:
prob_4 = prob_4.toarray().ravel()

In [78]:
temp_df["prob"] = prob_4

In [79]:
del prob_4

In [80]:
temp_df = temp_df[temp_df["prob"] >= 0.6]

In [81]:
temp_df.shape

(643082, 3)

In [82]:
final_df = pd.concat([final_df, temp_df])
del temp_df
final_df = final_df.reset_index().drop("index", axis = 1)

In [83]:
gc.collect()

0

In [84]:
temp_df = pd.DataFrame(columns = ["id", "terms", "prob"])

In [85]:
temp_df["id"] = [id_lst[i] for i in range(120000, 141865) for _ in range(1000)]

In [86]:
temp_df["terms"] = considered_one * 21865

In [87]:
with open('prob_5.pickle', 'rb') as f:
    prob_5 = pickle.load(f)

In [88]:
prob_5 = prob_5.toarray().ravel()

In [89]:
temp_df["prob"] = prob_5

In [90]:
del prob_5

In [91]:
temp_df = temp_df[temp_df["prob"] >= 0.6]

In [92]:
temp_df.shape

(474137, 3)

In [93]:
final_df = pd.concat([final_df, temp_df])
del temp_df
final_df = final_df.reset_index().drop("index", axis = 1)

In [94]:
final_df.shape

(3262524, 3)

In [95]:
# make the submission
final_df.to_csv(".../submission.tsv", index = False, sep = "\t")